<a href="https://colab.research.google.com/github/SegaObin/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers faiss-cpu

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Загрузка данных, данные через гптшку циклом нагенерил

In [ ]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/RAG/incidents.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

print(len(data))
data[0]

Объединю самари и описание проблемы, чтобы их было удобнее искать в векторной БД

In [ ]:
search_strings = [f'{item['summary']}. {item['description']}' for item in data]
search_strings[0]

In [ ]:
search_strings[3]

Делаю из этого эмбединги

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")
embeddings = model.encode(search_strings, show_progress_bar=True)

Импорчу векторную БД FAISS, и закидываю туда эмбединги всех проблем (саммари и описание проблемы)

In [ ]:
import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings.astype('float32'))

In [ ]:
index.d

Код для поиска по БД и считываение полной инфы о топ k проблемах из прошлого

In [ ]:
def find_similar_incedents(query_text, top_k=3):
  query_emb = model.encode([query_text]).astype('float32')
  distances, indices = index.search(query_emb, top_k)

  result = []
  for idx in indices[0]:
    incident = data[idx]
    result.append({
        'incident_id': incident['incident_id'],
        'summary': incident['summary'],
        'description': incident['description'],
        'resolution': incident['resolution']
    })

  return result

user_query = 'Закончилась память на диске'
similar_cases = find_similar_incedents(user_query)

In [ ]:
similar_cases

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Создание пропта для корректной работы RAG и понимания моедли че делать + настройка модели (температура, пенальти и тд) + обрезка предложения до точки на всякий случай потому что иногда модель генерит до талого

In [ ]:
import re

def generate_summary(query, similar_incidents):
    context = ""
    for i, inc in enumerate(similar_incidents[:3]):
        context += f"- {inc['resolution']}\n"

    prompt = f"""<|im_start|>system
Ты — лучший работник техподдержки. Твоя задача — дать максимально лаконичный ответ.
ЗАПРЕЩЕНО: использовать <think>, рассуждать, писать вступления.
Формат:
  1. НАПИШИ МИНИМУМ ТРИ ВАРИАНТА решения проблемы на основе опыта прошлых лет и твоих знаний.
  2. ОБЯЗАТЕЛЬНО пиши ответ в формате "Варианты решения вашей проблемы: 1).. 2).. 3).."
<|im_end|>
<|im_start|>user
Проблема с которой столкнулся пользователь: {query}
Опыт прошлых лет: {context}
<|im_end|>
<|im_start|>assistant
Варианты решения Вашей проблемы: """

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.2,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Извлекаем ответ
    answer = "Варианты решения Вашей проблемы: " + decoded_output.split("Варианты решения Вашей проблемы:")[-1].strip()

    # Очистка от мусора
    answer = re.sub(r'<think>.*?</think>', '', answer, flags=re.DOTALL)
    answer = answer.replace('</think>', '').strip()

    # ФИКС ЛОГИЧЕСКОГО ЗАВЕРШЕНИЯ:
    # Оставляем только текст до последней точки, чтобы убрать возможный обрыв
    if "." in answer:
        answer = answer[:answer.rfind(".")+1]

    return answer

Общая сборка всего, сначала находим похожие кейсы, генерим самари при помощи LLM, если хотм посмотреть ответ в ручном режиме то указываем silent=False это нужно чтобы нормлаьно реализовать проверку LLM-кой

In [ ]:
def process_new_incident(user_query, num_k, silent=True):
    similar_cases = find_similar_incedents(user_query, top_k=num_k)

    llm_advice = generate_summary(user_query, similar_cases)

    if not silent:
        print(f"=== ОТЧЕТ ДЛЯ K={num_k} ===")
        import pandas as pd
        display(pd.DataFrame(similar_cases))
        print("\nРЕКОМЕНДАЦИЯ:")
        print(llm_advice)
        print("-" * 30)
    else:
      return llm_advice

process_new_incident("Пользователи жалуются на медленную работу базы данных", num_k=5, silent=False)

Проверяю результат при помощи этой же LLM, но с другим промтов, для нескольких промтов пользователей по разным темам

In [ ]:
user_prompts = [
    'Очень быстро растет потребление памяти.',
    'Закончилось место на диске.',
    'База данных долго отвечает на запросы.',
]

In [ ]:
for query in user_prompts:

  model_ans_context_2 = process_new_incident(query, num_k=2)
  model_ans_context_5 = process_new_incident(query, num_k=5)
  model_ans_context_10 = process_new_incident(query, num_k=10)

  eval_prompt = f"""<|im_start|>system
Ты — аудитор технической поддержки. Сравни три ответа работника на запрос клиента в зависимости от того сколько похожих событий из прошлого сотрудник знает.
Запрос пользователя: {query}
Ответ 1 (сотрудник рассмотрел 2 инцидента): {model_ans_context_2}
Ответ 2 (сотрудник рассмотрел 5 инцидентов): {model_ans_context_5}
Ответ 3 (сотрудник рассмотрел 10 инцидентов): {model_ans_context_10}
Какой ответ более полный и точный? Оцени каждый ответ по шкале от одного до 5. Потом напиши итог какой ответ ты выбираешь как самый лучший.<|im_end|>
<|im_start|>assistant
Вот оценка каждого ответа сотрудника:
"""

  inputs_eval = tokenizer(eval_prompt, return_tensors="pt").to(device)

  outputs_eval = llm_model.generate(
      **inputs_eval,
      max_new_tokens=250,      # Уменьшаем лимит, чтобы не было места для дублей
      temperature=0.4,        # Минимальная температура для исключения повторов
      repetition_penalty=1.3, # Повышаем штраф за повторение одних и тех же слов
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id
  )

  decoded_output_eval = tokenizer.decode(outputs_eval[0], skip_special_tokens=True)

  answer_eval = "Вот оценка каждого ответа сотрудника: " + decoded_output_eval.split("Вот оценка каждого ответа сотрудника:")[-1].strip()

  if "." in answer_eval:
      answer_eval = answer_eval[:answer_eval.rfind(".")+1]

  print(10 * '=' + f'ОЦЕНКА РЕЗУЛЬТАТА ДЛЯ ПРОМПТА "{query}"' + 50 * '=')
  print(answer_eval)
  print(60 * '=')
  print('\n\n')